In [1]:
import json
import spacy
import en_core_web_lg
nlp = spacy.load("en_core_web_lg")
import string
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import classification_report, make_scorer
from random import shuffle
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
import scipy.stats
import eli5
from langdetect import detect
import re
from spacy.tokenizer import Tokenizer
from spacy.lang.en.tokenizer_exceptions import TOKENIZER_EXCEPTIONS

/home/liudmyla/.local/lib/python3.8/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/liudmyla/.local/lib/python3.8/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
!cat 11-0.txt 1342-0.txt 84-0.txt > all.txt

In [54]:
with open('all.txt') as f:
    text = f.read()

In [55]:
text = text.replace('\n', ' ').replace('—', '').replace('_', '').replace('’',"'").replace('”','"').replace('“','"')
text = text.replace('  ',' ').replace('   ',' ').replace('    ',' ').replace('     ',' ').replace('      ',' ')
t1 = text[:1000000]
sents1 = []
doc = nlp(t1)
for sent in doc.sents:
    if sent.string.strip()[-1] in ['!', '.', '?'] and sent.string.strip()[0].istitle():
        sents1.append(sent.string.strip())
t2 = text[1000000:]
sents2 = []
doc = nlp(t2)
for sent in doc.sents:
    if sent.string.strip()[-1] in ['!', '.', '?'] and sent.string.strip()[0].istitle():
        sents2.append(sent.string.strip())
sents = sents1 + sents2

In [56]:
len(sents)

6590

In [58]:
# У тексті багато мов, залишаю речення англійською
en = []
for sent in sents:
    if detect(sent)=='en':
        en.append(sent)
len(en)/len(sents)

0.9660091047040971

# Формую датасет

In [59]:
# sent1 - буде 1 речення, sent2 - 2 склеєні речення і так далі
sent2 = en[: int(len(en) * .70)]
sent3 = en[int(len(en) * .70):int(len(en) * .95)]
sent1 = en[int(len(en) * .95):int(len(en) * .975)]
sent4 = en[int(len(en) * .975):]

In [60]:
def decapitalize(text):
    if len(text.split(' ')[0])>1:
        res = [text.split(' ')[0].lower()]+text.split(' ')[1:]
    else:
        res = [text.split(' ')[0]]+text.split(' ')[1:]
    return ' '.join(res)

In [61]:
def depunctuate(text):
    res = text.split(' ')[:-1]+[text.split(' ')[-1].strip(string.punctuation)]
    return ' '.join(res)

In [62]:
def label(text):
    sent = []
    doc = nlp(text)
    for token in doc:
        if token in doc[:-1]:
            sent.append([token.text, False])
        else:
            sent.append([token.text, True])
    return sent

In [63]:
def label_p(text):
    sent = []
    doc = nlp(text)
    for token in doc:
        sent.append([token.text, False])
    return sent

In [64]:
dsent2 = [(decapitalize(x) if i % 4 == 1 else x) for i, x in enumerate(sent2)]
dp2 = [(depunctuate(x) if i%2==0 else x) for i, x in enumerate(dsent2)]
lab2 = [(label(x) if i%2==0 else label_p(x)) for i, x in enumerate(dp2)]
labeled_2 = [x+y for x,y in zip(*[iter(lab2)]*2)]

In [65]:
dsent3 = [(decapitalize(x) if i % 5 == 0 else x) for i, x in enumerate(sent3)]
dp3 = [(x if i in range(2, len(dsent3)+1, 3) else depunctuate(x)) for i, x in enumerate(dsent3)]
lab3 = [(label_p(x) if i in range(2, len(dsent3)+1, 3) else label(x)) for i, x in enumerate(dp3)]
labeled_3 = [x+y+z for x,y,z in zip(*[iter(lab3)]*3)]

In [66]:
dsent4 = [(decapitalize(x) if i % 8 == 1 else x) for i, x in enumerate(sent4)]
dp4 = [(x if i in range(3, len(dsent4)+1, 4) else depunctuate(x)) for i, x in enumerate(dsent4)]
lab4 = [(label_p(x) if i in range(3, len(dsent4)+1, 4) else label(x)) for i, x in enumerate(dp4)]
labeled_4 = [x+y+z+e for x,y,z,e in zip(*[iter(lab4)]*4)]

In [67]:
labeled_1 = [label_p(x) for x in sent1]

In [68]:
data = labeled_1 + labeled_2+ labeled_3+labeled_3
shuffle(data)
len(data)

3447

In [70]:
data[0]

[['It', False],
 ['was', False],
 ['situated', False],
 ['against', False],
 ['the', False],
 ['back', False],
 ['of', False],
 ['the', False],
 ['cottage', False],
 ['and', False],
 ['surrounded', False],
 ['on', False],
 ['the', False],
 ['sides', False],
 ['which', False],
 ['were', False],
 ['exposed', False],
 ['by', False],
 ['a', False],
 ['pig', False],
 ['sty', False],
 ['and', False],
 ['a', False],
 ['clear', False],
 ['pool', False],
 ['of', False],
 ['water', True],
 ['I', False],
 ['had', False],
 ['first', False],
 [',', False],
 ['however', False],
 [',', False],
 ['provided', False],
 ['for', False],
 ['my', False],
 ['sustenance', False],
 ['for', False],
 ['that', False],
 ['day', False],
 ['by', False],
 ['a', False],
 ['loaf', False],
 ['of', False],
 ['coarse', False],
 ['bread', False],
 [',', False],
 ['which', False],
 ['I', False],
 ['purloined', False],
 [',', False],
 ['and', False],
 ['a', False],
 ['cup', False],
 ['with', False],
 ['which', False],
 ['I',

In [71]:
def custom_tokenizer(nlp):
    hyphen_re = re.compile(r'[A-Za-z\d]+-[A-Za-z\d]+')
    
    prefixes = list(nlp.Defaults.prefixes)
    prefixes.remove('"')
    prefix_re = spacy.util.compile_prefix_regex(prefixes)
    infix_re = spacy.util.compile_infix_regex(nlp.Defaults.infixes)
    suffixes = list(nlp.Defaults.suffixes)
    suffixes.remove('"')
    suffix_re = spacy.util.compile_suffix_regex(suffixes)

    return Tokenizer(nlp.vocab, rules=TOKENIZER_EXCEPTIONS, prefix_search=prefix_re.search, infix_finditer=infix_re.finditer,
                         suffix_search=suffix_re.search, token_match=hyphen_re.match)

nlp.tokenizer = custom_tokenizer(nlp)
contracted_verbs = ['’m','’re','’s','’t','’ve','’d','’ll','n’t',"'m","'re","'s","'t","'ve","'d","'ll","n't"]
def sent2features(sent):
    tokens = []
    for i in range(len(sent)):
        tokens.append(sent[i][0])
    text = ''.join([('' if c in contracted_verbs else ' ')+c for c in tokens]).strip()
    doc = nlp(text)
    res = [(token.text, token.pos_, token.dep_, token.head.text, token.head.pos_, doc.get_lca_matrix()[i], token.n_lefts, token.n_rights) for i, token in enumerate(doc)]
    return res

In [72]:
sent2features(data[0])[0]

('It',
 'PRON',
 'nsubjpass',
 'situated',
 'VERB',
 array([ 0,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
         2,  2,  2,  2,  2,  2,  2,  2,  2,  2, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1], dtype=int32),
 0,
 0)

In [73]:
def sent2labels(sent):
    return [label for token, label in sent]

In [74]:
%%time
train, test = train_test_split(clean_data, test_size=0.3, random_state=42)
train_tokens = [sent2features(s) for s in train]
train_labels = [sent2labels(s) for s in train]

test_tokens = [sent2features(s) for s in test]
test_labels = [sent2labels(s) for s in test]

CPU times: user 13min 4s, sys: 3.18 s, total: 13min 7s
Wall time: 13min 7s


In [85]:
print(len(train_tokens)==len(train_labels))
print(len(test_tokens)==len(test_labels))
train1 = [len(x) for x in train_tokens]
train1.sort()
train2 = [len(x) for x in train_labels]
train2.sort()
test1 = [len(x) for x in test_tokens]
test1.sort()
test2 = [len(x) for x in test_labels]
test2.sort()
print(train1==train2)
print(test1==test2)

True
True
True
True


In [84]:
# del train_tokens[901]
# del train_labels[901]
# del test_tokens[940]
# del test_labels[940]

In [76]:
# a = [len(x) for x in train_tokens]
# b = [len(x) for x in train_labels]
# [(x,y) for x,y in list(zip(a,b)) if x!=y]

[(45, 44)]

# Baseline

In [86]:
# проставити всім токенам клас False
def classify(token):
    res = []
    for i in range(len(token)):
        res.append(False)     
    return res

In [87]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
test_labels_bin = mlb.fit_transform(test_labels)

In [88]:
predicted = [classify(x) for  x in test_tokens]
print(classification_report(test_labels_bin, mlb.transform(predicted)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1007
           1       0.00      0.00      0.00       956

   micro avg       1.00      0.51      0.68      1963
   macro avg       0.50      0.50      0.50      1963
weighted avg       0.51      0.51      0.51      1963
 samples avg       1.00      0.53      0.68      1963



/home/liudmyla/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# CRF 

In [89]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    dep = sent[i][2]
    head = sent[i][3]
    head_pos = sent[i][4]
    lca = sent[i][5]
    left_ch = sent[i][6]
    right_ch = sent[i][7]
    
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'dep': dep,
        'head': head,
        'head_pos': head_pos,
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        dep1 = sent[i-1][2]
        head1 = sent[i-1][3]
        head_pos1 = sent[i-1][4]
        
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:dep': dep1,
            '-1:head': head1,
            '-1:head_pos': head_pos1,
            'left_ch': left_ch,
        })
    else:
        features['BOS'] = True
        
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        dep1 = sent[i+1][2]
        head1 = sent[i+1][3]
        head_pos1 = sent[i+1][4]
        
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:dep': dep1,
            '+1:head': head1,
            '+1:head_pos': head_pos1,
            '+1:lca': sent[lca[i+1]][0],
            'right_ch': right_ch,
        })
    else:
        features['EOS'] = True
        
    if i > 1:
        word1 = sent[i-2][0]
        postag1 = sent[i-2][1]
        dep1 = sent[i-2][2]
        head1 = sent[i-2][3]
        head_pos1 = sent[i-2][4]
        
        features.update({
            '-2:word.lower()': word1.lower(),
            '-2:word.istitle()': word1.istitle(),
            '-2:word.isupper()': word1.isupper(),
            '-2:postag': postag1,
            '-2:dep': dep1,
            '-2:head': head1,
            '-2:head_pos': head_pos1,
        })
        
    if i < len(sent)-2:
        word1 = sent[i+2][0]
        postag1 = sent[i+2][1]
        dep1 = sent[i+2][2]
        head1 = sent[i+2][3]
        head_pos1 = sent[i+2][4]
        
        features.update({
            '+2:word.lower()': word1.lower(),
            '+2:word.istitle()': word1.istitle(),
            '+2:word.isupper()': word1.isupper(),
            '+2:postag': postag1,
            '+2:dep': dep1,
            '+2:head': head1,
            '+2:head_pos': head_pos1,
            '+2:lca': sent[lca[i+2]][0],
        })
    
    return features

In [90]:
def sent2features2(sent):
    return [word2features(sent, i) for i in range(len(sent))]

In [91]:
%%time
X_train = [sent2features2(s) for s in train_tokens]
y_train = train_labels

X_test = [sent2features2(s) for s in test_tokens]
y_test = test_labels

CPU times: user 773 ms, sys: 7.88 ms, total: 781 ms
Wall time: 778 ms


In [92]:
def bool_to_string(labels):
    return [str(x) for x in labels]

In [93]:
y_train = [bool_to_string(x) for x in y_train]
y_test = [bool_to_string(x) for x in y_test]

In [94]:
%%time
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', 
    c1=0.1, 
    c2=0.1, 
    max_iterations=100, 
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

CPU times: user 12.5 s, sys: 7.95 ms, total: 12.5 s
Wall time: 12.5 s


/home/liudmyla/.local/lib/python3.8/site-packages/sklearn/base.py:193: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

In [95]:
eli5.show_weights(crf, top=30)

/home/liudmyla/.local/lib/python3.8/site-packages/sklearn/base.py:193: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


In [96]:
labels = list(crf.classes_)
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred, 
                      average='weighted', labels=labels)

0.9932815209538947

In [97]:
print(metrics.flat_classification_report(
    y_test, y_pred, labels=labels, digits=3
))

              precision    recall  f1-score   support

       False      0.995     0.998     0.997     54995
        True      0.922     0.779     0.845      1266

    accuracy                          0.994     56261
   macro avg      0.959     0.889     0.921     56261
weighted avg      0.993     0.994     0.993     56261



# CRF + perplexity + prob

In [98]:
# модель звідси https://openslr.org/11/
import kenlm
model = kenlm.Model('/tank/liudmyla/4-gram.mmap')
print(model.perplexity('LANGUAGE MODELING IS FUN'))
print(model.perplexity('LANGUAGE MODELING IS'))

5056.315352560162
7988.458056921622


In [99]:
print(model.perplexity('I LIKE BEER'))
print(model.perplexity('I LIKE BEER MORNING'))

112.93413049481077
707.3608785933942


In [100]:
# якщо перплексіті без останнього слова зменшується - ставимо флаг тру для попереднього слова
def shorter_less_perplex(words):
    sentence = ' '.join(words).upper()
    words2 = words[:-1]
    shorter_sentence = ' '.join(words2).upper()
    longer = model.perplexity(sentence)/len(words)
    shorter = model.perplexity(shorter_sentence)/(len(words2))
    return shorter < longer

In [101]:
def two_vs_one(words):
    assert len(words) > 1
    shorter = (model.score(' '.join(words[:-1]).upper()) + model.score(words[-1].upper()))/len(words)
    longer = model.score(' '.join(words).upper())/len(words)
    return shorter < longer

In [102]:
w = "i like beer morning".split()
shorter_less_perplex(w), two_vs_one(w)

(True, True)

In [103]:
import math
def prob(words):
    phrase = ' '.join(words).upper()
    return math.exp(model.score(phrase))

In [104]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    dep = sent[i][2]
    head = sent[i][3]
    head_pos = sent[i][4]
    lca = sent[i][5]
    left_ch = sent[i][6]
    right_ch = sent[i][7]
    
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'dep': dep,
        'head': head,
        'head_pos': head_pos,
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        dep1 = sent[i-1][2]
        head1 = sent[i-1][3]
        head_pos1 = sent[i-1][4]
        
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:dep': dep1,
            '-1:head': head1,
            '-1:head_pos': head_pos1,
            'left_ch': left_ch,
        })
    else:
        features['BOS'] = True
        
    for s in (slice(i-3,i+2),slice(i-2,i+2), slice(i-1,i+2)):
        words = sent[s]
        if len(words) > 2:
            features[f'{repr(s)}:shorter-less-perplex'] = shorter_less_perplex([w[0] for w in words])
            features[f'{repr(s)}:two-vs-one'] = two_vs_one([w[0] for w in words])

    for back in (3, 2, 1):
        words = sent[i-back:i+2]
        if len(words) > 2:
            features[f'{back}:shorter-less-perplex'] = shorter_less_perplex([w[0] for w in words])
            features[f'{back}:two-vs-one'] = two_vs_one([w[0] for w in words])


    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        dep1 = sent[i+1][2]
        head1 = sent[i+1][3]
        head_pos1 = sent[i+1][4]
        
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:dep': dep1,
            '+1:head': head1,
            '+1:head_pos': head_pos1,
            '+1:lca': sent[lca[i+1]][0],
            'right_ch': right_ch,
            '+1:prob': prob([sent[i][0],sent[i+1][0]])
        })
    else:
        features['EOS'] = True
        
    if i > 1:
        word1 = sent[i-2][0]
        postag1 = sent[i-2][1]
        dep1 = sent[i-2][2]
        head1 = sent[i-2][3]
        head_pos1 = sent[i-2][4]
        
        
        features.update({
            '-2:word.lower()': word1.lower(),
            '-2:word.istitle()': word1.istitle(),
            '-2:word.isupper()': word1.isupper(),
            '-2:postag': postag1,
            '-2:dep': dep1,
            '-2:head': head1,
            '-2:head_pos': head_pos1,
        })
        
    if i < len(sent)-2:
        word1 = sent[i+2][0]
        postag1 = sent[i+2][1]
        dep1 = sent[i+2][2]
        head1 = sent[i+2][3]
        head_pos1 = sent[i+2][4]
        
        features.update({
            '+2:word.lower()': word1.lower(),
            '+2:word.istitle()': word1.istitle(),
            '+2:word.isupper()': word1.isupper(),
            '+2:postag': postag1,
            '+2:dep': dep1,
            '+2:head': head1,
            '+2:head_pos': head_pos1,
            '+2:lca': sent[lca[i+2]][0],
            '+2:prob': prob([sent[i][0],sent[i+1][0],sent[i+2][0]])
        })
    if i > 2:
        word1 = sent[i-3][0]
        postag1 = sent[i-3][1]
        dep1 = sent[i-3][2]
        head1 = sent[i-3][3]
        head_pos1 = sent[i-3][4]
        
        
        features.update({
            '-3:word.lower()': word1.lower(),
            '-3:word.istitle()': word1.istitle(),
            '-3:word.isupper()': word1.isupper(),
            '-3:postag': postag1,
            '-3:dep': dep1,
            '-3:head': head1,
            '-3:head_pos': head_pos1,
        })
    if i < len(sent)-3:
        word1 = sent[i+3][0]
        postag1 = sent[i+3][1]
        dep1 = sent[i+3][2]
        head1 = sent[i+3][3]
        head_pos1 = sent[i+3][4]
        
        features.update({
            '+3:word.lower()': word1.lower(),
            '+3:word.istitle()': word1.istitle(),
            '+3:word.isupper()': word1.isupper(),
            '+3:postag': postag1,
            '+3:dep': dep1,
            '+3:head': head1,
            '+3:head_pos': head_pos1,
            '+3:lca': sent[lca[i+3]][0],
            '+3:prob': prob([sent[i][0],sent[i+1][0],sent[i+2][0],sent[i+3][0]])
        })
    
    return features

In [105]:
train[15]

[['Girls', False],
 [',', False],
 ['can', False],
 ['I', False],
 ['do', False],
 ['anything', False],
 ['for', False],
 ['you', False],
 ['in', False],
 ['Meryton', True],
 ['Here', False],
 ['comes', False],
 ['Hill', False],
 ['!', False]]

In [106]:
sent2features2(train_tokens[15])[1]

{'bias': 1.0,
 'word.lower()': ',',
 'word.isupper()': False,
 'word.istitle()': False,
 'word.isdigit()': False,
 'postag': 'PUNCT',
 'dep': 'punct',
 'head': 'do',
 'head_pos': 'AUX',
 '-1:word.lower()': 'girls',
 '-1:word.istitle()': True,
 '-1:word.isupper()': False,
 '-1:postag': 'NOUN',
 '-1:dep': 'npadvmod',
 '-1:head': 'do',
 '-1:head_pos': 'AUX',
 'left_ch': 0,
 'slice(0, 3, None):shorter-less-perplex': True,
 'slice(0, 3, None):two-vs-one': True,
 '1:shorter-less-perplex': True,
 '1:two-vs-one': True,
 '+1:word.lower()': 'can',
 '+1:word.istitle()': False,
 '+1:word.isupper()': False,
 '+1:postag': 'VERB',
 '+1:dep': 'aux',
 '+1:head': 'do',
 '+1:head_pos': 'AUX',
 '+1:lca': 'do',
 'right_ch': 0,
 '+1:prob': 0.00041450226405163583,
 '+2:word.lower()': 'i',
 '+2:word.istitle()': True,
 '+2:word.isupper()': True,
 '+2:postag': 'PRON',
 '+2:dep': 'nsubj',
 '+2:head': 'do',
 '+2:head_pos': 'AUX',
 '+2:lca': 'do',
 '+2:prob': 0.00019373693522132932,
 '+3:word.lower()': 'do',
 '+3:

In [107]:
def sent2features2(sent):
    return [word2features(sent, i) for i in range(len(sent))]

In [108]:
%%time
X_train = [sent2features2(s) for s in train_tokens]
y_train = train_labels

X_test = [sent2features2(s) for s in test_tokens]
y_test = test_labels

CPU times: user 8.01 s, sys: 52 ms, total: 8.06 s
Wall time: 8.06 s


In [109]:
def bool_to_string(labels):
    return [str(x) for x in labels]

In [110]:
y_train = [bool_to_string(x) for x in y_train]
y_test = [bool_to_string(x) for x in y_test]

In [111]:
%%time
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', 
    c1=0.1, 
    c2=0.1, 
    max_iterations=100, 
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

CPU times: user 23.6 s, sys: 20.2 ms, total: 23.7 s
Wall time: 23.7 s


CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

In [112]:
eli5.show_weights(crf, top=30)

/home/liudmyla/.local/lib/python3.8/site-packages/sklearn/base.py:193: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


In [113]:
print(metrics.flat_classification_report(
    y_test, y_pred, labels=labels, digits=3
))

              precision    recall  f1-score   support

       False      0.995     0.998     0.997     54995
        True      0.922     0.779     0.845      1266

    accuracy                          0.994     56261
   macro avg      0.959     0.889     0.921     56261
weighted avg      0.993     0.994     0.993     56261



In [114]:
labels = list(crf.classes_)
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred, 
                      average='weighted', labels=labels)

0.9939647274330626

# Гіпепараметри

In [115]:
%%time
# define fixed parameters and parameters to search
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', 
    max_iterations=100, 
    all_possible_transitions=True
)
params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score, 
                        average='weighted', labels=labels)

# search
rs = RandomizedSearchCV(crf, params_space, 
                        cv=3, 
                        verbose=1, 
                        n_jobs=-1, 
                        n_iter=50, 
                        scoring=f1_scorer)
rs.fit(X_train, y_train)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  38 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed: 10.4min finished


CPU times: user 2min 57s, sys: 12 s, total: 3min 9s
Wall time: 10min 51s


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=CRF(algorithm='lbfgs', all_possible_states=None,
                                 all_possible_transitions=True, averaging=None,
                                 c=None, c1=None, c2=None,
                                 calibration_candidates=None,
                                 calibration_eta=None,
                                 calibration_max_trials=None,
                                 calibration_rate=None,
                                 calibration_samples=None, delta=None,
                                 epsilon=None, error_sensitive=None, gamma=None,
                                 keep_...
                   iid='deprecated', n_iter=50, n_jobs=-1,
                   param_distributions={'c1': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f10b97f6580>,
                                        'c2': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f10b4946370>},
                

In [116]:
print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

best params: {'c1': 0.42239424788837804, 'c2': 0.02655614993898217}
best CV score: 0.9935789433034325
model size: 0.43M


In [117]:
crf = rs.best_estimator_
y_pred = crf.predict(X_test)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=labels, digits=3
))

              precision    recall  f1-score   support

       False      0.996     0.998     0.997     54995
        True      0.920     0.813     0.863      1266

    accuracy                          0.994     56261
   macro avg      0.958     0.906     0.930     56261
weighted avg      0.994     0.994     0.994     56261



In [118]:
eli5.show_weights(crf, top=30)

/home/liudmyla/.local/lib/python3.8/site-packages/sklearn/base.py:193: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


# Тестові дані

In [119]:
with open('run-on-test.json') as f:
    hold = json.load(f)

In [120]:
hold_tokens = [sent2features(s) for s in hold]
hold_tokens = [sent2features2(s) for s in hold_tokens]
hold_labels = [sent2labels(s) for s in hold]
hold_labels = [bool_to_string(x) for x in hold_labels]

In [121]:
print(len(hold_tokens)==len(hold_labels))
hold1 = [len(x) for x in hold_tokens]
hold1.sort()
hold2 = [len(x) for x in hold_labels]
hold2.sort()
print(hold1==hold2)

True
True


In [122]:
hold_pred = crf.predict(hold_tokens)
metrics.flat_f1_score(hold_labels, hold_pred, 
                      average='weighted', labels=labels)

0.9790499974124188

In [123]:
print(metrics.flat_classification_report(
    hold_labels, hold_pred, labels=labels, digits=3
))

              precision    recall  f1-score   support

       False      0.988     0.990     0.989      4542
        True      0.699     0.658     0.678       155

    accuracy                          0.979      4697
   macro avg      0.843     0.824     0.834      4697
weighted avg      0.979     0.979     0.979      4697

